# Comandos do MYSQL



### Para inserir algo:

In [ ]:
db('''
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (1, 'Juca', 'Silva'), 
    (2, 'Mario', 'Ferreira'), 
    (3, 'Ana', 'Soares'), 
    (4, 'Antonio', 'Reis'), 
    (5, 'Paulo', 'Oliveira')
''')

Achar aquele que tem vários VALUES.

### Para iniciar uma transação: 

In [ ]:
db('START TRANSACTION')
db('INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES (8, "Carlos", "Sainz")')
db('COMMIT')

# Posso reverter a transação em uma outra transação

db('START TRANSACTION')
db('''
SELECT id_usuario INTO @id_usuario 
    FROM usuario 
    WHERE nome = 'Barack' AND sobrenome = 'Trump' 
    ORDER BY id_usuario DESC LIMIT 1;
''')
db('SELECT @id_usuario')
db('DELETE FROM usuario WHERE id_usuario=@id_usuario')
db('COMMIT')

### Para criar funções para transação seguir o padrão:



In [ ]:
def deposito(connection, id_usuario, valor):
    with connection.cursor() as cursor:
        # Atualizei o saldo da pessoa depois do depósito
        query1 = '''
        UPDATE 
            usuario
        SET
            saldo = saldo + %s 
        WHERE
            id_usuario = %s
        '''
        params1 = (valor, id_usuario)
        cursor.execute(query1, params1)
        
        # Log da movimentação de conta corrente.
        
        query2 = '''
        INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)
        '''
        params2 = (id_usuario, valor)
        
        cursor.execute(query2, params2)

In [ ]:
def pega_id_usuario(connection, nome, sobrenome):
    with connection.cursor() as cursor:
        cursor.execute(
            '''
            SELECT id_usuario 
            FROM usuario 
            WHERE nome = %s AND sobrenome = %s''', (nome, sobrenome))

        results = cursor.fetchall()
        
    if results is None:
        raise KeyError(f'Usuario {nome} {sobrenome} não encontrado.')
    
    return results[0][0]


# retorna último id inserido
cursor.execute('SELECT LAST_INSERT_ID()')


In [ ]:
def atualiza_emprestimo(connection, id_emprestimo, valor):
    with connection.cursor() as cursor:
        # Atualiza saldo do emprestimo.
        query = '''
        UPDATE 
            emprestimo
        SET 
            valor_atual = valor_atual + %s
        WHERE 
            id_emprestimo = %s
        '''
        params = (valor, id_emprestimo)
        cursor.execute(query, params)

        # Log na tabela de operações.
        query = '''
        INSERT INTO operacao (id_emprestimo, valor) VALUES (%s, %s)
        '''
        params = (id_emprestimo, valor)
        cursor.execute(query, params)

### Para testar funções de transação:
    

In [ ]:
dados = [
    ("Ana", "Soares", "Juca", "Silva", 1000),
    ("Ana", "Soares", "Antonio", "Reis", 2000),
    ("Paulo", "Oliveira", "Juca", "Silva", 3000),
]

for nome_credor, sobrenome_credor, \
    nome_devedor, sobrenome_devedor, \
    valor in dados:
    try:
        start_transaction(connection)
        id_credor = pega_id_usuario(
            connection,
            nome_credor,
            sobrenome_credor,
        )
        id_devedor = pega_id_usuario(
            connection,
            nome_devedor,
            sobrenome_devedor,
        )
        id_emprestimo = cria_emprestimo_(connection, id_credor, id_devedor)
        saque(connection, id_credor, valor)
        deposito(connection, id_devedor, valor)
        atualiza_emprestimo(connection, id_emprestimo, valor)
        commit(connection)
    except Exception as e:
        print(e)
        rollback(connection)

### Para criar uma procedure:

``` mysql

USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

DELIMITER //
CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome) VALUES (novo_nome, novo_sobrenome);
END//
DELIMITER ;

```

``` mysql

USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30,2), IN id_usuario_procurado INT)
BEGIN
    DECLARE taxa_cobrada DECIMAL(30,2);
    SELECT IF(saldo(id_usuario_procurado)>0.0, taxa, 0.0) INTO taxa_cobrada;
    UPDATE usuario SET saldo = saldo - taxa WHERE id_usuario = id_usuario_procurado;
END//
DELIMITER ;

```

Note o uso destes comandos DELIMITER. Coisas de MySQL: ele não entende que os ponto-e-virgula internos ao procedimento não sinalizam o final do comando CREATE PROCEDURE...

### Como chamar e usar procedures: CALL.

Observação: prefira trabalhar com isso no Workbench, é melhor. Já teve bug uma vez.

In [ ]:
db('START TRANSACTION')
try:
    db("CALL adiciona_usuario('Max', 'Verstappen');")
    db("CALL adiciona_usuario('Lando', 'Norris');")
    db("CALL adiciona_usuario('Charles', 'Leclerc');")
except Exception as e:
    print(e)
    db('ROLLBACK');

- Para criar uma function:

``` mysql
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE saldo_procurado DECIMAL(30, 2);
    SELECT IFNULL(saldo, 0.0) INTO saldo_procurado FROM usuario WHERE id_usuario = id;
    RETURN saldo_procurado;
END//
DELIMITER ;

COMMIT;
```


``` mysql

USE emprestimos;

DROP FUNCTION IF EXISTS total_saldo;

DELIMITER //
CREATE FUNCTION total_saldo() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE soma_total DECIMAL(30, 2);
    SELECT 
        SUM(saldo) INTO soma_total 
    FROM usuario;
    RETURN soma_total;
END//
DELIMITER ;

```

### Para usar uma função e chamá-la:

In [ ]:
db("SELECT saldo(1)")

OBS2: Prefira rodar no MySql Workbench os testes. Caso esteja "hanging", simplesmente feche a conexão do notebook e espere rodar tudo no Workbench. Depois abra de novo o jupyter e siga a vida :)

### Funções do MySQL:

https://dev.mysql.com/doc/refman/8.0/en/flow-control-functions.html#function_if

- IF:

```mysql
mysql> SELECT IF(1>2,2,3);
        -> 3
mysql> SELECT IF(1<2,'yes','no');
        -> 'yes'
mysql> SELECT IF(STRCMP('test','test1'),'no','yes');
        -> 'no'
```

- IFNULL:


``` mysql

mysql> SELECT IFNULL(1,0);
        -> 1
mysql> SELECT IFNULL(NULL,10);
        -> 10
mysql> SELECT IFNULL(1/0,10);
        -> 10
mysql> SELECT IFNULL(1/0,'yes');
        -> 'yes'
```

### Para criar um trigger:

Trigger simples que toda vez que tenho uma movimentação atualiza o saldo do usuário. Ou seja, posta sempre o conteúdo na movimentação:

``` mysql

USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;

```

OBS: Se for DELETE, precisa usar o .OLD

Nesse contexto a operação é basicamente: já existe um empréstimo, vou aumentar mais. Ou seja, estou criando uma operação que vai agir atualizando o empréstimo que já existe e criando movimentações. E como a movimentação já tem o trigger anterior, também atualiza o saldo do usuário.


``` mysql

USE emprestimos;

DROP TRIGGER IF EXISTS trig_operacao;

DELIMITER //
CREATE TRIGGER trig_operacao
BEFORE INSERT ON operacao
FOR EACH ROW
BEGIN
	UPDATE emprestimo 
		SET valor_atual = valor_atual + NEW.valor # valor da operacao - aumentando a dívida. Precisamos inserir mov 
        WHERE id_emprestimo = NEW.id_emprestimo;
	SELECT id_credor INTO @id_credor
		FROM emprestimo 
		WHERE id_emprestimo = NEW.id_emprestimo;
	SELECT id_devedor INTO @id_devedor
		FROM emprestimo 
		WHERE id_emprestimo = NEW.id_emprestimo;
	INSERT INTO movimentacao (id_usuario, valor) VALUES (@id_credor, -NEW.valor);
	INSERT INTO movimentacao (id_usuario, valor) VALUES (@id_devedor, NEW.valor);
END //

DELIMITER ;

```


### Para testar o Trigger:

```mysql
USE emprestimos;

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

START TRANSACTION;

INSERT INTO movimentacao (id_usuario, valor) VALUES(1, 50000);

INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2);
INSERT INTO operacao (id_emprestimo, valor) VALUES (LAST_INSERT_ID(), 2000);

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

ROLLBACK;

SELECT * FROM usuario;
SELECT * FROM emprestimo;
SELECT * FROM operacao;

```

### Para adicionar uma constraint de verificação de valor:

CONSTRAINT c_saldo CHECK (saldo >= 0.0)

``` mysql


CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

```

### Para criar uma VIEW:

Lista nome e sobrenome de usuários sem revelar saldos: 
```mysql
USE emprestimos;

CREATE VIEW nomes AS 
    SELECT DISTINCT nome, sobrenome FROM usuario;
```

Porque usar views? Assim como no caso de stored procedures, podemos configurar permissões de acesso diferentes para esta view. Suponha que um vendedor deva ter acesso aos nomes dos clientes, mas não aos seus saldos (por razões de confidencialidade). Podemos conceder ao vendedor acesso apenas à essa view. Poderíamos ter resolvido o problema também com uma stored procedure: em SQL as coisas costumam ter várias soluções possíveis...

```mySQL

USE emprestimos

CREATE VIEW devedor AS
    SELECT 
        id_usuario, SUM(IFNULL(valor_atual, 0.0)) as total
    FROM 
        usuario
        LEFT OUTER JOIN emprestimo ON usuario.id_usuario = emprestimo.id_devedor
    GROUP BY
        id_usuario
```

```mySQL

USE emprestimos

CREATE VIEW valor_liquido AS
    SELECT 
        id_usuario, nome, sobrenome, saldo + credor.total - devedor.total as valor
    FROM 
        usuario
        INNER JOIN credor USING (id_usuario)
        INNER JOIN devedor USING (id_usuario)
```

```mySQL

USE emprestimos

CREATE VIEW valor_liquido AS
    SELECT 
        id_usuario, nome, sobrenome, saldo + credor.total - devedor.total as valor
    FROM 
        usuario
        INNER JOIN credor USING (id_usuario)
        INNER JOIN devedor USING (id_usuario)
```